In [3]:
import pandas as pd
import numpy as np
from pyjet import cluster,DTYPE_PTEPM
from sklearn import preprocessing
from scipy.stats import iqr
import tensorflow as tf

# Pre processing

In [1]:
path = '/anomalyvol/data/events_LHCO2020_backgroundMC_Pythia.h5'

In [23]:
df = pd.read_hdf(path,stop=100) # just read first 10000 events

In [25]:
all_events = df.values

In [26]:
rows = all_events.shape[0]
cols = all_events.shape[1]
data = []
# more_data
data2 = []
print(rows, cols)

100 2100


In [51]:
nparts = []
for i in range(rows):
    pseudojets_input = np.zeros(len([x for x in all_events[i][::3] if x > 0]), dtype=DTYPE_PTEPM)
    for j in range(cols // 3):
        if (all_events[i][j*3]>0):
            pseudojets_input[j]['pT'] = all_events[i][j*3]
            pseudojets_input[j]['eta'] = all_events[i][j*3+1]
            pseudojets_input[j]['phi'] = all_events[i][j*3+2]
        pass
    # cluster jets from the particles in one observation
    sequence = cluster(pseudojets_input, R=1.0, p=-1)
    jets = sequence.inclusive_jets()
    for k in range(len(jets)): # for each jet get (px, py, pz, e)
        if jets[k].pt < 200: continue
        particles = np.zeros((100,4))
        nparts.append(len(jets[k]))
        for p, part in enumerate(jets[k]):
            if len(jets[k])==1: print(part)
            if p >= 100: break
            particles[p,:] = np.array([part.px,
                                       part.py,
                                       part.pz,
                                       part.e])
        data.append(particles)
print(data)
print(np.mean(nparts))
print(np.max(nparts))
print(np.min(nparts))
print(np.percentile(nparts, 95))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [55]:
data = np.stack(data,axis=0)
print(data.shape)
print(data)

(5891, 100, 4)
[[[-4.53540316 -1.26489676  6.25380246  7.82814735]
  [-1.29560759 -0.02328982  1.41958818  1.92207493]
  [-0.68179651 -1.17327202  0.73898564  1.54515808]
  ...
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]]

 [[ 0.15441286  0.31261169  0.13826326  0.3750815 ]
  [ 2.21352603  0.95378217  2.51937375  3.48663764]
  [ 0.81016925  0.43543171  0.40410066  1.00462547]
  ...
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]]

 [[ 4.95538828 -1.92603623 11.14609147 12.34912319]
  [ 0.40725781  0.1808598   2.26965288  2.31298366]
  [ 1.68009995  0.60353411  4.741078    5.0660448 ]
  ...
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]]

 ...

 [[ 1.38188861 -0.1

In [65]:
# splitting data
x_train = data[0:int(len(data) * 0.8)]
x_val = data[int(len(data) * 0.8):int(len(data) * 0.9)]
x_test = data[int(len(data) * 0.9):]

In [64]:
# regularize (skip for now)
scaler = preprocessing.StandardScaler()
x_train_reshape = scaler.fit_transform(x_train.reshape((-1,4)))
x_val_reshape = scaler.transform(x_val.reshape((-1,4)))
x_test_reshape = scaler.transform(x_test.reshape((-1,4)))
x_train = x_train_reshape.reshape(-1,100,4)
x_val = x_val_reshape.reshape(-1,100,4)
x_test = x_test_reshape.reshape(-1,100,4)
print(x_train)

[[[-2.72299269e-01 -8.21571044e-02  2.56643247e-01  1.26510648e-01]
  [-7.78589666e-02 -1.15153018e-03  5.36558854e-02 -5.43751421e-02]
  [-4.10203428e-02 -7.61792750e-02  2.50775672e-02 -6.59190070e-02]
  ...
  [-1.01489351e-04  3.67956843e-04 -5.95224188e-03 -1.13242697e-01]
  [-1.01489351e-04  3.67956843e-04 -5.95224188e-03 -1.13242697e-01]
  [-1.01489351e-04  3.67956843e-04 -5.95224188e-03 -1.13242697e-01]]

 [[ 9.16578765e-03  2.07635337e-02 -1.46604528e-04 -1.01755043e-01]
  [ 1.32745988e-01  6.25951163e-02  9.98355867e-02 -6.45714497e-03]
  [ 4.85218115e-02  2.87766219e-02  1.10158367e-02 -8.24739459e-02]
  ...
  [-1.01489351e-04  3.67956843e-04 -5.95224188e-03 -1.13242697e-01]
  [-1.01489351e-04  3.67956843e-04 -5.95224188e-03 -1.13242697e-01]
  [-1.01489351e-04  3.67956843e-04 -5.95224188e-03 -1.13242697e-01]]

 [[ 2.97302212e-01 -1.25291514e-01  4.62069151e-01  2.64974965e-01]
  [ 2.43405872e-02  1.21677074e-02  8.93498742e-02 -4.24027468e-02]
  [ 1.00731768e-01  3.97440474e-

In [75]:
# make graph representation

import keras
from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization, Dropout, Lambda, Conv1D, SpatialDropout1D, Concatenate, Flatten, Reshape
import keras.backend as K
from keras.backend import slice

def conv(nconvs, nconvinputs):

    inputs_conv = Input(shape=(nconvs, nconvinputs,), name = 'input_conv')

    y = Conv1D(filters=64, kernel_size=(1,), strides=(1,), padding='same', 
               kernel_initializer='he_normal', use_bias=True, name='conv_1',
               activation = 'relu')(inputs_conv)
    y = Conv1D(filters=32, kernel_size=(1,), strides=(1,), padding='same', 
               kernel_initializer='he_normal', use_bias=True, name='conv_2',
               activation = 'relu')(y)
    y = Conv1D(filters=32, kernel_size=(1,), strides=(1,), padding='same', 
               kernel_initializer='he_normal', use_bias=True, name='conv_3',
               activation = 'relu')(y)

    y = Lambda(lambda x: K.mean(x, axis=-2), name = 'lambda_1', input_shape=(nconvs,32)) (y) 
                 
    y = Dense(128, name = 'dense_1', activation='relu', kernel_initializer='glorot_uniform')(y)
    y = Dense(nconvs*nconvinputs, name = 'dense_2', activation='linear', kernel_initializer='glorot_uniform')(y)
    outputs = Reshape((nconvs, nconvinputs), name = 'output')(y)

    keras_model = Model(inputs=inputs_conv, outputs=outputs)
    
    return keras_model
    
keras_model = conv(100, 4)
print(keras_model.summary())

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_conv (InputLayer)      (None, 100, 4)            0         
_________________________________________________________________
conv_1 (Conv1D)              (None, 100, 64)           320       
_________________________________________________________________
conv_2 (Conv1D)              (None, 100, 32)           2080      
_________________________________________________________________
conv_3 (Conv1D)              (None, 100, 32)           1056      
_________________________________________________________________
lambda_1 (Lambda)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               4224      
_________________________________________________________________
dense_2 (Dense)              (None, 400)               5160